<a href="https://colab.research.google.com/github/ghostrider0007/Spam_Email_Detection/blob/main/Spam_Email_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/completeSpamAssassin.csv")
df.head()

,Unnamed: 0,Body,Label
0,0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,3,##############################################...,1
4,4,I thought you might like these:\n1) Slim Down ...,1


In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df.rename(columns={'Label': 'spam','Body':'Message'}, inplace=True)

In [ ]:
df.head()

,Message,spam
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,I thought you might like these:\n1) Slim Down ...,1


In [ ]:
df.spam.value_counts()

0    4150
1    1896
Name: spam, dtype: int64

In [ ]:
df.shape

(6046, 2)

In [ ]:
df.head()

,Message,spam
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,I thought you might like these:\n1) Slim Down ...,1


In [ ]:
df.isnull().sum()

Message    1
spam       0
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Message, df.spam, test_size=0.2)

In [ ]:
X_train.shape

(4836,)

In [ ]:
X_test.shape

(1209,)

In [ ]:
type(X_train)

pandas.core.series.Series

In [ ]:
X_train[:4]

2216    right Mike,i will agree to disagree but i take...
2126    Hi,some time now the following messages were h...
2434    >>>>> "G" == Geege Schuman  writes:    G> ... ...
2606    Clearly, it is US/NATO = Sun/IBM/OSS, USSR = M...
Name: Message, dtype: object

In [ ]:
type(y_train)

pandas.core.series.Series

In [ ]:
y_train[:4]

2216    0
2126    0
2434    0
2606    0
Name: spam, dtype: int64

In [ ]:
type(X_train.values)

numpy.ndarray

In [ ]:
X_train.head()

2216    right Mike,i will agree to disagree but i take...
2126    Hi,some time now the following messages were h...
2434    >>>>> "G" == Geege Schuman  writes:    G> ... ...
2606    Clearly, it is US/NATO = Sun/IBM/OSS, USSR = M...
3910    URL: http://boingboing.net/#85497383\nDate: No...
Name: Message, dtype: object

<h3>Create bag of words representation using CountVectorizer</h3>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

X_train_cv = v.fit_transform(X_train.values)
X_train_cv

<4836x53724 sparse matrix of type '<class 'numpy.int64'>'
	with 648953 stored elements in Compressed Sparse Row format>

In [ ]:
X_train_cv.toarray()[:2][0]

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
X_train_cv.shape

(4836, 53724)

In [ ]:
v.get_feature_names_out()[1771]

'194'

In [ ]:
v.vocabulary_

In [ ]:
X_train_np = X_train_cv.toarray()
X_train_np[0]

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
np.where(X_train_np[0]!=0)

(array([  327,   569,   977,  1938,  1968,  3440,  4489,  6839,  7105,
         7156,  7224,  7544,  7599,  7773,  7844,  7915,  7948,  7987,
         7990,  8080,  8117,  8125,  8186,  8255,  8340,  8352,  8465,
         8757,  8788,  8957,  9066,  9170,  9368,  9539,  9593,  9700,
         9859, 10111, 10168, 10177, 10181, 10216, 10296, 10351, 10470,
        10544, 10592, 10905, 11225, 11489, 11764, 11850, 11880, 11904,
        11928, 12191, 12193, 12225, 12269, 12385, 12398, 12446, 12560,
        12563, 12651, 12754, 12826, 12918, 12920, 12927, 12969, 13894,
        13974, 14084, 14132, 14138, 14199, 14416, 14780, 14809, 14909,
        15088, 15094, 15112, 15113, 15403, 15475, 15671, 15801, 15944,
        16061, 16098, 16284, 16286, 16547, 17085, 17121, 17162, 17319,
        17736, 17793, 17797, 17889, 18052, 18066, 18120, 18138, 18431,
        18520, 18563, 18574, 18807, 19054, 19073, 19429, 19948, 19955,
        19972, 19982, 20289, 20318, 20450, 20536, 20658, 20673, 21177,
      

<h3>Train the naive bayes model</h3>

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_cv, y_train)

MultinomialNB()

In [ ]:
X_test_cv = v.transform(X_test)

<h3>Evaluate Performance</h3>

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_cv)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97       833
           1       0.89      0.97      0.93       376

    accuracy                           0.95      1209
   macro avg       0.94      0.96      0.95      1209
weighted avg       0.96      0.95      0.96      1209



In [ ]:
emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]

emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1])

In [ ]:
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [ ]:
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [ ]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97       833
           1       0.89      0.97      0.93       376

    accuracy                           0.95      1209
   macro avg       0.94      0.96      0.95      1209
weighted avg       0.96      0.95      0.96      1209

